In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
cc = CasosConfirmados()
# cc.read()
# cc.save()
cc.load()
notifica = Notifica()
# notifica.download_todas_notificacoes()
# notifica.save()
notifica.load()
print(cc.shape())
print(notifica.shape())

(586944, 10728, 432271, 143945)
(2310261, 1061440, 11001, 1235648, 2172)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['duplicado']==0) | ((casosn['duplicado']==1) & (casosn['manter']==1))]
print(casosn.shape)

(2310261, 55)


In [4]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

(28108, 23)

In [5]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.shape

(651673, 55)

In [6]:
#merge hash_resid
casos_hash_resid = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])]
print(f"casos_hash_resid {len(casos_hash_resid)}")
notifica_hash_resid = casosn.loc[(casosn['hash_resid'].isin(casosc['hash_resid']))]
print(f"notifica_hash_resid {len(notifica_hash_resid)}")
merge_hash_resid = pd.merge(casos_hash_resid,notifica_hash_resid[['hash_resid','id','manter']])
print(f"merge_hash_resid {len(merge_hash_resid)}")
merge_hash_resid = merge_hash_resid.drop_duplicates('ordem')
print(f"merge_hash_resid {len(merge_hash_resid)}\n")

#merge hash_atend
casos_hash_atend = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])]
print(f"casos_hash_atend {len(casos_hash_atend)}")
notifica_hash_atend = casosn.loc[(casosn['hash_atend'].isin(casosc['hash_atend']))]
print(f"notifica_hash_atend {len(notifica_hash_atend)}")
merge_hash_atend = pd.merge(casos_hash_atend,notifica_hash_atend[['hash_atend','id','manter']])
print(f"merge_hash_atend {len(merge_hash_atend)}")
merge_hash_atend = merge_hash_atend.drop_duplicates('ordem')
print(f"merge_hash_atend {len(merge_hash_atend)}\n")

#merge hash_diag
casos_hash_diag = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])]
print(f"casos_hash_diag {len(casos_hash_diag)}")
notifica_hash_diag = casosn.loc[(casosn['hash_diag'].isin(casosc['hash_diag']))]
print(f"notifica_hash_diag {len(notifica_hash_diag)}")
merge_hash_diag = pd.merge(casos_hash_diag,notifica_hash_diag[['hash_diag','id','manter']])
print(f"merge_hash_diag {len(merge_hash_diag)}")
merge_hash_diag = merge_hash_diag.drop_duplicates('ordem')
print(f"merge_hash_diag {len(merge_hash_diag)}\n")

#trasnforma essa lista de index em um conjunto, para deixar apenas os index unicos
idx_casos_confirmados = set(casos_hash_resid.index.tolist() + casos_hash_atend.index.tolist() + casos_hash_diag.index.tolist())
print(f"idx_casos_confirmados {len(idx_casos_confirmados)}")

#realiza a diferença de conjunto para pegar os index dos que não foram encotrados no notifica
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
print(f"casos_confirmados_nao_notifica {len(casos_confirmados_nao_notifica)}") ## aqui o numero total é 27910 casos não encontrados no notifica
len(idx_casos_confirmados) + len(casos_confirmados_nao_notifica)

casos_hash_resid 537408
notifica_hash_resid 631946
merge_hash_resid 631946
merge_hash_resid 537408

casos_hash_atend 506822
notifica_hash_atend 578944
merge_hash_atend 585317
merge_hash_atend 506822

casos_hash_diag 478054
notifica_hash_diag 480465
merge_hash_diag 486454
merge_hash_diag 478054

idx_casos_confirmados 558836
casos_confirmados_nao_notifica 28108


In [13]:
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).drop_duplicates('ordem').sort_values('ordem')
merge_casos_notifica = merge_casos_notifica[['ordem', 'identificacao', 'id', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'obito', 'data_obito', 'data_com_obito', 'recuperado', 'ativo', 'excluir']]

,ordem,identificacao,ibge_resid,ibge_atend,paciente,sexo,idade,mun_resid,mun_atend,rs,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,obito,data_obito,data_com_obito,recuperado,ativo,excluir,hash_resid,hash_atend,hash_diag,id,manter
7,33,3827,4113700,4113700,ARTHUR BUFFARA VAN DEN BERG,M,37,LONDRINA,LONDRINA,17,SABIN,2020-03-20,2020-03-20,NaT,0,NaT,NaT,1,0,NaN,ARTHURBUFFARAVANDENBERG37LONDRINA,ARTHURBUFFARAVANDENBERG37LONDRINA,ARTHURBUFFARAVANDENBERG20032020,501391.0,1.0
24,84,6946,4115200,4115200,DANILLO RONQUI,M,32,MARINGA,MARINGA,15,LACEN,2020-03-25,2020-03-25,2020-03-13,0,NaT,NaT,1,0,NaN,DANILLORONQUI32MARINGA,DANILLORONQUI32MARINGA,DANILLORONQUI25032020,186175.0,1.0
31,109,10853,4109401,4109401,FERNANDO JOSE COSTA,M,57,GUARAPUAVA,GUARAPUAVA,5,LACEN,2020-03-26,2020-03-27,2020-03-08,1,2020-06-15,NaT,0,0,NaN,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA57GUARAPUAVA,FERNANDOJOSECOSTA26032020,83233.0,1.0
40,126,23026,4118402,4118402,MARTA CLEMENTINA SAMPAIO,F,62,PARANAVAI,PARANAVAI,14,LACEN,2020-03-27,2020-03-28,2020-03-22,0,NaT,NaT,1,0,NaN,MARTACLEMENTINASAMPAIO62PARANAVAI,MARTACLEMENTINASAMPAIO62PARANAVAI,MARTACLEMENTINASAMPAIO27032020,82542.0,1.0
44,141,4382,4106209,4106209,BRUNA SCHLICHTING,F,30,CONTENDA,CONTENDA,2,LACEN,2020-03-28,2020-03-29,2020-03-14,0,NaT,NaT,1,0,NaN,BRUNASCHLICHTING30CONTENDA,BRUNASCHLICHTING30CONTENDA,BRUNASCHLICHTING28032020,80394.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631910,586911,668661,4127700,4127700,WILLIAM QUEIROZ LOPES,M,20,TOLEDO,TOLEDO,20,TESTE MOLECULAR (RT-PCR),2021-02-12,2021-02-13,2021-02-06,0,NaT,NaT,0,1,NaN,WILLIAMQUEIROZLOPES20TOLEDO,WILLIAMQUEIROZLOPES20TOLEDO,WILLIAMQUEIROZLOPES12022021,2372007.0,1.0
631913,586913,668663,4115200,4115200,WILLIAM ROBERTO VARGAS SERRA,M,48,MARINGA,MARINGA,15,TESTE RAPIDO IGM/IGG NAO DISCRIMINATORIO (IMUN...,2021-01-09,2021-02-13,NaT,0,NaT,NaT,1,0,NaN,WILLIAMROBERTOVARGASSERRA48MARINGA,WILLIAMROBERTOVARGASSERRA48MARINGA,WILLIAMROBERTOVARGASSERRA09012021,2372792.0,1.0
631915,586914,668664,4101002,4101002,WILLIAM WILSOM SIRTOLI BARBIERI,M,22,AMPERE,AMPERE,8,COVID-19 BIOLOGIA MOLECULAR,2021-02-12,2021-02-13,2021-02-05,0,NaT,NaT,0,1,NaN,WILLIAMWILSOMSIRTOLIBARBIERI22AMPERE,WILLIAMWILSOMSIRTOLIBARBIERI22AMPERE,WILLIAMWILSOMSIRTOLIBARBIERI12022021,2325475.0,1.0
631922,586917,668667,4119905,4119905,WILLIAN RAMOS GARCZAREK,M,29,PONTA GROSSA,PONTA GROSSA,3,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-02-12,2021-02-13,NaT,0,NaT,NaT,0,1,NaN,WILLIANRAMOSGARCZAREK29PONTAGROSSA,WILLIANRAMOSGARCZAREK29PONTAGROSSA,WILLIANRAMOSGARCZAREK12022021,2377614.0,1.0


In [8]:
writer = pd.ExcelWriter("merge_casos_notifica.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

merge_casos_notifica.to_excel(writer,sheet_name='geral',index=None)

writer.save()
writer.close()

SystemExit: 